# OmniMarket Strategy Builder
Simulate multi-leg options strategies like spreads, straddles, and condors.

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
# User Input
ticker = input("Enter ticker (e.g. AAPL): ").upper()
yf_ticker = yf.Ticker(ticker)
expirations = yf_ticker.options
print("Available Expiration Dates:", expirations)
expiry = input("Select an expiration date (copy from above): ")

In [ ]:
# Multi-leg Strategy Builder
chain = yf_ticker.option_chain(expiry)
calls, puts = chain.calls, chain.puts
legs = []
print("\nDefine your strategy legs one by one. Type 'done' when finished.")
while True:
    leg_type = input("Call/Put (type 'done' to finish): ").lower()
    if leg_type == 'done': break
    strike = float(input("Strike price: "))
    side = input("Buy or Sell: ").lower()
    legs.append({'type': leg_type, 'strike': strike, 'side': side})

In [ ]:
# Simulation
S0 = yf_ticker.history(period='1d')['Close'].iloc[-1]
price_range = np.linspace(S0 * 0.7, S0 * 1.3, 150)
pnl = np.zeros_like(price_range)
for leg in legs:
    df = calls if leg['type'] == 'call' else puts
    row = df[df['strike'] == leg['strike']].iloc[0]
    premium = (row['bid'] + row['ask']) / 2
    for i, price in enumerate(price_range):
        value = max(0, price - leg['strike']) if leg['type'] == 'call' else max(0, leg['strike'] - price)
        leg_pnl = value - premium if leg['side'] == 'buy' else premium - value
        pnl[i] += leg_pnl

In [ ]:
# Plotting
plt.figure(figsize=(10, 5))
plt.plot(price_range, pnl, label='Strategy P&L')
plt.axhline(0, color='black', linestyle='--')
plt.xlabel('Stock Price at Expiry')
plt.ylabel('Profit / Loss')
plt.title(f'{ticker} Multi-leg Strategy P&L')
plt.grid(True)
plt.legend()
plt.show()